# Segmented Representations

One common representation in evolutionary algorithms (EA) is that of a "segmented representation."  That is, each individual is comprised of a sequence of segments, which are themselves fixed-length sequences, and are usually binary, but needn't be.  Each segment represents a salient feature, such as a rule in a Pitt Approach system, or a convolutional layer and its hyperparameters, as is the case for Multi-node Evolutionary Neural Networks for Deep Learning (MENNDL).

There are two broad categories for these systems: those that have a fixed number of such segments, as is the case currently for MENNDL, and a dynamic number of segments, as is the case for Pitt Approach classifiers.

In this notebook we look at LEAP support for segmented representations, starting with initializers and decoders, and then looking at the mutation pipeline operator.  We then plug all that into a simple EA example.


In [2]:
import sys
import random
import functools
import pprint

from leap_ec.individual import Individual

from leap_ec.segmented_rep.initializers import create_segmented_sequence
from leap_ec.segmented_rep.decoders import SegmentedDecoder
from leap_ec.segmented_rep.ops import apply_mutation

from leap_ec.binary_rep.initializers import create_binary_sequence
from leap_ec.binary_rep.ops import genome_mutate_bitflip
from leap_ec.binary_rep.decoders import BinaryToIntDecoder

from leap_ec.real_rep.initializers import create_real_vector
from leap_ec.real_rep.ops import genome_mutate_gaussian

## Binary genomes

We first look at segmented representations with segments that use a binary representaion.

In [3]:
# Create a genome of four segments of five binary digits.
seg = create_segmented_sequence(4, create_binary_sequence(5))
print(seg)

[[1, 0, 0, 1, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]


In [4]:
# Now create five genomes of varying length by passing in a function for `length` that provides an
# integer drawn from a distribution.
seqs = [] # Save sequences for next step
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=5), create_binary_sequence(5))
    print(i, seq)
    seqs.append(seq)

0 [[1, 0, 0, 1, 0], [1, 0, 0, 1, 1], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]
1 [[0, 1, 0, 0, 1], [1, 0, 1, 0, 0], [1, 0, 0, 1, 0]]
2 [[1, 0, 1, 1, 1], [0, 0, 1, 1, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 0], [1, 1, 1, 0, 1]]
3 [[0, 0, 1, 0, 1]]
4 [[1, 1, 1, 0, 1], [1, 1, 1, 0, 0]]


Now let's see about decoding those segments.  The segmented representation relies on a secondary decoder that's applied to each segment.  In this case, we'll just use a simple binary to int decoder on the segments we created in the previous step.

In [5]:
# We want each segment to have two integers from the five bits.
decoder = SegmentedDecoder(BinaryToIntDecoder(2,3)) 

for i, seq in enumerate(seqs):
    vals = decoder.decode(seq)
    print(i, vals)

0 [[2, 2], [2, 3], [1, 6], [3, 6], [1, 3]]
1 [[1, 1], [2, 4], [2, 2]]
2 [[2, 7], [0, 6], [2, 1], [1, 6], [3, 5]]
3 [[0, 5]]
4 [[3, 5], [3, 4]]


In [7]:
# And now for mutation, which shows that, on average, a single value is changed in an example individual.  The
# takeaway here is that segmented mutation just uses a mutator from another representation and naively applies it.

original = Individual([[0,0],[1,1]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=genome_mutate_bitflip))
print('mutated:', mutated)

original: [[0, 0], [1, 1]]
mutated: [[1, 0], [0, 1]]


## Real-valued genomes

Now we demonstrate the same process using a real-valued representation.

In [8]:
# Create five segmented sequences that vary from 1 to 3 segments
bounds = ((-5.12,5.12), (-1,1), (-10,10)) # three reals and their respective bounds for sampling
seqs = []
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=3), 
                                    create_real_vector(bounds))
    seqs.append(seq)

# Just for fun, now add a genome that has exactly 5 segments
seqs.append(create_segmented_sequence(5, create_real_vector(bounds)))

for i, s in enumerate(seqs):
    print(i, pprint.pformat(s, indent=2))

0 [[2.5818863135384253, -0.37558069879717126, 9.099497101642271]]
1 [ [0.24939945503415295, -0.7337440882709738, 7.221552801236374],
  [3.00448723484016, -0.6723027991741866, 3.1146440287158583],
  [2.794586843451172, -0.8401371966971041, 0.29549660427013436]]
2 [[3.083466554786022, -0.7198359825669398, -7.3288063085215]]
3 [ [-0.16765014084703367, 0.3466521940061895, 1.4419995335302094],
  [2.6579264936306117, 0.7240124147743956, 0.7421377716995625]]
4 [ [-4.404115853703007, -0.48011675272171517, -1.233842635380375],
  [-3.7973733923684154, 0.22973430568006892, 1.2976782307971018]]
5 [ [-3.912039117858524, 0.3453415846561778, 0.10425673029951099],
  [1.5415924909270737, -0.9868263068376215, 9.233553597785843],
  [-0.8952549413887683, -0.8358921683093938, -8.377552852997782],
  [0.7567281001893784, -0.5029369603639975, -0.9602945008812185],
  [-4.120030242221674, 0.5363453442342894, -7.324892091491142]]


Now we repeat the application of the segmented mutation operator, but this time to real-valued genomes.

In [9]:
original = Individual([[0.0,0.0],[1.0,1.0],[-1.0,0.0]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=genome_mutate_gaussian(std=1.0)))
print('mutated:', mutated)

original: [[0.0, 0.0], [1.0, 1.0], [-1.0, 0.0]]


NameError: name 'expected_num_mutation' is not defined